In [1]:
from dstruct import DataStructFromJSON, DataField, datafield

In [2]:
class AccountSummaryFromFile(DataStructFromJSON):
    
    user = DataField(str)
    type = DataField(str, 'account', 'account-type')
    ballance = DataField(float, 'account', 'account-ballance')
    # you can pass functions to `dtype` for simple data parsing or type coersion
    account_number = DataField(lambda s: 'X'*len(s[:-4])+s[-4:], 'account', 'account-number')

In [3]:
print(AccountSummaryFromFile('bank_data.json'))

AccountSummaryFromFile(
    "account_number": "XXXXX6789",
    "ballance": 1234.56,
    "type": "checking",
    "user": "John F. Doe"
)


In [4]:
from dstruct import DataStruct
from datetime import datetime

In [5]:
class Transaction(DataStruct):
    
    amount = DataField(float)
    
    # use the `datafield` decorator for
    # more complex parsing functions
    @datafield(dict, path=None)
    def time(self, data):
        s = data['utc-unix']
        tz = data['time-zone']
        if 'UTC' not in tz:
            raise ValueError("Unknow time-zone standard: '%s'" % tz)
        else:
            dif = tz.replace('UTC', '')
            # trick for adding time-zone
            s = eval(str(s)+dif+'*60*60')

        dt = datetime.fromtimestamp(s)
        # return the parsed epoch time
        # for the given time-zone
        return dt.strftime('%Y-%m-%d %H:%M:%S')
    
    @datafield(dict, 'source')
    def source(self, data):
        t = data['type']
        if '-' in t:
            s = ''
            for sub in t.split('-'):
                s += sub.capitalize()
        else:
            s = t.capitalize()

        cls = eval(s)
        return cls(data)
    

class TransactionSource(DataStruct):
    
    ref = DataField(str)
    
class Purchase(TransactionSource):
    
    at = DataField(str, 'name')
    card = DataField(lambda s: 'X'*len(s[:-4])+s[-4:])
    
class MobileDeposit(TransactionSource):
    
    note = DataField(str)
    check_number = DataField(str, 'check-number')


class DetailedAccountSummary(AccountSummaryFromFile):
    
    last_withdraw = DataField(Transaction, 'account', 'withdrawn', '0')
    last_deposit = DataField(Transaction, 'account', 'deposited', '0')

In [6]:
print(DetailedAccountSummary('bank_data.json'))

DetailedAccountSummary(
    "account_number": "XXXXX6789",
    "ballance": 1234.56,
    "last_deposit": {
        "amount": 1057.21,
        "source": {
            "check_number": "1229361",
            "note": "bi-weekly paycheck",
            "ref": " #IB3GFRZG31"
        },
        "time": "2016-03-08 06:34:51"
    },
    "last_withdraw": {
        "amount": 1057.21,
        "source": {
            "at": "Average-Restaurant",
            "card": "XXXXXXXXXXXX1112",
            "ref": "S567013305806010"
        },
        "time": "2016-03-08 06:34:51"
    },
    "type": "checking",
    "user": "John F. Doe"
)


In [21]:
import inspect
import sys

In [24]:
def repr_type(obj):
    """ Return a string representation of a value and its type for readable
    error messages.
    """
    the_type = type(obj)
    if (not py3compat.PY3) and the_type is InstanceType:
        # Old-style class.
        the_type = obj.__class__
    msg = '%r %r' % (obj, the_type)
    return msg

In [25]:
repr_type(1)

"1 <type 'int'>"

In [26]:
class A(object):
    
    a = 'a'

NameError: name 'a' is not defined